In [1]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import torch

In [2]:
# Data path 
data_path = "../Test_Functions/data/bangkok_traffy.csv"

# Load data
Traffyticket = pd.read_csv(data_path)

In [3]:
Traffyticket.head()

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
0,2021-9LHDM6,{},NaN,ไม่มีภาพ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.48661,13.79386",1867 จรัญสนิทวงศ์ แขวง บางพลัด เขตบางพลัด กรุง...,บางพลัด,บางพลัด,กรุงเทพมหานคร,2021-09-01 10:44:55.353209+00,กำลังดำเนินการ,NaN,NaN,2022-02-22 04:59:58.622268+00
1,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-09-03 12:51:09.453003+00,เสร็จสิ้น,NaN,NaN,2022-06-04 15:34:14.609206+00
2,2021-8GKAR9,{สายไฟ},ยังไม่มีหน่วยงานรับผิดชอบ,1. เถาวัลย์งอดบนสายไฟ ผมกลัวว่าต่อไปจะเอาออกยา...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.57685,13.79704",335/31 ลาดพร้าว แขวง สามเสนนอก เขตห้วยขวาง กรุ...,สามเสนนอก,ห้วยขวาง,จังหวัดกรุงเทพมหานคร,2021-09-19 06:47:50.488685+00,กำลังดำเนินการ,NaN,NaN,2022-02-22 04:30:07.293416+00
3,2021-AFPUXZ,"{ถนน,สะพาน}",NaN,1 ซ่อมทางเท้าหลังคืนพื้นที่จากการปรับปรุงสะพาน...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.52916,13.72338",37 10 ซอย สีลม 9 แขวง สีลม เขตบางรัก กรุงเทพมห...,สีลม,บางรัก,กรุงเทพมหานคร,2021-09-19 07:40:30.000781+00,กำลังดำเนินการ,NaN,NaN,2022-02-22 04:30:04.507406+00
4,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,NaN,2022-06-21 08:21:09.532782+00


In [4]:
Traffyticket.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267969 entries, 0 to 267968
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ticket_id      265368 non-null  object 
 1   type           267872 non-null  object 
 2   organization   266971 non-null  object 
 3   comment        265368 non-null  object 
 4   photo          267875 non-null  object 
 5   photo_after    180804 non-null  object 
 6   coords         267969 non-null  object 
 7   address        265368 non-null  object 
 8   subdistrict    267897 non-null  object 
 9   district       267895 non-null  object 
 10  province       267944 non-null  object 
 11  timestamp      267969 non-null  object 
 12  state          267969 non-null  object 
 13  star           98143 non-null   float64
 14  count_reopen   150268 non-null  float64
 15  last_activity  267969 non-null  object 
dtypes: float64(2), object(14)
memory usage: 32.7+ MB


In [5]:
Traffyticket.describe()

,star,count_reopen
count,98143.000000,150268.000000
mean,3.851716,0.185475
std,1.435982,0.754284
min,1.000000,0.000000
25%,3.000000,0.000000
50%,4.000000,0.000000
75%,5.000000,0.000000
max,5.000000,63.000000


In [6]:
Traffyticket.shape

(267969, 16)

In [7]:
print(f"Number of rows: {Traffyticket.shape[0]}")
# Filter to collect ticket which state is 'เสร็จสิ้น'
Traffyticket = Traffyticket[Traffyticket['state'] == 'เสร็จสิ้น']
Traffyticket.head(3)

# Filter to remove nan value
print("Dropping nan value....")
Traffyticket = Traffyticket.dropna()    
print(f"Number of rows: {Traffyticket.shape[0]}")

Number of rows: 267969
Dropping nan value....
Number of rows: 47589


In [8]:
Traffyticket.columns

Index(['ticket_id', 'type', 'organization', 'comment', 'photo', 'photo_after',
       'coords', 'address', 'subdistrict', 'district', 'province', 'timestamp',
       'state', 'star', 'count_reopen', 'last_activity'],
      dtype='object')

In [9]:
drop_columns = ['photo', 'photo_after', 'ticket_id', 'coords', 'address', 'comment', 'state', 'last_activity', 'timestamp', 'star', 'subdistrict', 'organization', 'count_reopen']

# Calculate by convert last_activity and timestamp to datetime and calculate to add the duration column
Traffyticket['last_activity'] = pd.to_datetime(Traffyticket['last_activity'])
Traffyticket['timestamp'] = pd.to_datetime(Traffyticket['timestamp'])
Traffyticket['duration'] = Traffyticket['last_activity'] - Traffyticket['timestamp']

# Convert duration to days
Traffyticket['duration'] = Traffyticket['duration'].dt.days

# Show the result
Traffyticket.head(3)

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity,duration
764,2022-48FAE2,{สะพาน},"อบต.แอตแลนติส (ใช้ทดสอบ),เขตราชเทวี,ฝ่ายรักษาค...",เห็นเศษไม้ เศษก้อนปูนทิ้งไว้ข้างสะพานหัวช้าง ร...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53138,13.74958",80 ถ. พญาไท แขวง ถนนเพชรบุรี เขตราชเทวี กรุงเท...,ถนนเพชรบุรี,ราชเทวี,กรุงเทพมหานคร,2022-05-26 06:59:10.781756+00:00,เสร็จสิ้น,5.0,1.0,2022-07-25 06:41:26.471152+00:00,59
786,2022-L4HGDX,{},"เขตคลองเตย,ฝ่ายเทศกิจ เขตคลองเตย,ผอ.เขตคลองเตย...",ติดตามเรื่องที่แจ้ง,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.59469,13.71168",6 ถ. สุขุมวิท แขวง พระโขนง เขตคลองเตย กรุงเทพม...,พระโขนง,คลองเตย,กรุงเทพมหานคร,2022-05-27 03:42:31.051225+00:00,เสร็จสิ้น,5.0,1.0,2022-06-24 07:57:10.339134+00:00,28
787,2022-A7QR9P,"{ถนน,สะพาน}","เขตลาดกระบัง,สำนักการโยธา กทม.,สำนักงานก่อสร้า...",กทม.ช่วยปรับปรุงถนนกทม.ให้ดีและวางแผนปรับพื้นใ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.78370,13.73035",4/56 ถนน ฉลองกรุง แขวง ลำปลาทิว เขตลาดกระบัง ก...,ลำปลาทิว,ลาดกระบัง,กรุงเทพมหานคร,2022-05-27 04:02:38.305569+00:00,เสร็จสิ้น,1.0,1.0,2023-04-26 13:21:47.513069+00:00,334


In [10]:
# Drop the columns
Traffyticket.drop(drop_columns, axis=1, inplace=True)
# Reset index
Traffyticket.reset_index(drop=True, inplace=True)
# Show the result
Traffyticket.head(3)

,type,district,province,duration
0,{สะพาน},ราชเทวี,กรุงเทพมหานคร,59
1,{},คลองเตย,กรุงเทพมหานคร,28
2,"{ถนน,สะพาน}",ลาดกระบัง,กรุงเทพมหานคร,334


In [11]:
print(Traffyticket.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47589 entries, 0 to 47588
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   type      47589 non-null  object
 1   district  47589 non-null  object
 2   province  47589 non-null  object
 3   duration  47589 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.5+ MB
None


In [12]:
Traffyticket

,type,district,province,duration
0,{สะพาน},ราชเทวี,กรุงเทพมหานคร,59
1,{},คลองเตย,กรุงเทพมหานคร,28
2,"{ถนน,สะพาน}",ลาดกระบัง,กรุงเทพมหานคร,334
3,{},หลักสี่,กรุงเทพมหานคร,158
4,"{คลอง,ถนน}",ตลิ่งชัน,กรุงเทพมหานคร,254
...,...,...,...,...
47584,{ถนน},ลาดกระบัง,กรุงเทพมหานคร,0
47585,{},จอมทอง,กรุงเทพมหานคร,0
47586,{},มีนบุรี,กรุงเทพมหานคร,0
47587,{น้ำท่วม},บางกอกน้อย,กรุงเทพมหานคร,0


In [13]:
def extract_types(df):
    result = []
    for index,row in Traffyticket.iterrows():
        types = row['type'].strip('{}').split(',')
        for t in types: 
            new_row = row.copy()
            new_row['type'] = t.strip()
            result.append(new_row)
    return result

In [14]:
Traffyticket = pd.DataFrame(extract_types(Traffyticket))

Traffyticket.head(10)

,type,district,province,duration
0,สะพาน,ราชเทวี,กรุงเทพมหานคร,59
1,,คลองเตย,กรุงเทพมหานคร,28
2,ถนน,ลาดกระบัง,กรุงเทพมหานคร,334
2,สะพาน,ลาดกระบัง,กรุงเทพมหานคร,334
3,,หลักสี่,กรุงเทพมหานคร,158
4,คลอง,ตลิ่งชัน,กรุงเทพมหานคร,254
4,ถนน,ตลิ่งชัน,กรุงเทพมหานคร,254
5,น้ำท่วม,บางแค,กรุงเทพมหานคร,94
5,ถนน,บางแค,กรุงเทพมหานคร,94
5,ท่อระบายน้ำ,บางแค,กรุงเทพมหานคร,94


## Prepare the data before train the model

In [15]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')

nominal_columns = ['type', 'district', 'province']

# Fit the encoder
enc.fit(Traffyticket[nominal_columns])

# Transform the categorical columns to numerical columns
enc_cols = enc.transform(Traffyticket[nominal_columns]).toarray()

# Create the new datafram with the encoded columns
enc_df = pd.DataFrame(enc_cols, columns=enc.get_feature_names_out(nominal_columns))

# Merge the original dataframe with the encoded dataframe
Traffyticket = enc_df.join(Traffyticket)

# Drop the original categorical columns 
Traffyticket.drop(nominal_columns, axis=1, inplace=True)

# from sklearn.preprocessing import OneHotEncoder, StandardScaler
# from sklearn.compose import ColumnTransformer

# # Define which columns should be one-hot encoded
# categorical_cols = ['type', 'district', 'province']

# # Define which columns should be scaled
# numeric_cols = ['count_reopen']

# # Create transformers for each column type
# categorical_transformer = OneHotEncoder(handle_unknown='ignore')
# numeric_transformer = StandardScaler()

# # Combine the transformers into a single preprocessor
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('cat', categorical_transformer, categorical_cols),
#         ('num', numeric_transformer, numeric_cols)
#     ])

# # Fit the preprocessor on the training data
# preprocessor.fit(X_train)

# # Transform the training and testing data
# X_train = preprocessor.transform(X_train)
# X_test = preprocessor.transform(X_test)


In [16]:
print(f"Shape of the dataframe before dropping: {Traffyticket.shape}")

# Drop NaN value
Traffyticket.dropna(inplace=True)

print(f"Shape of the dataframe after dropping: {Traffyticket.shape}")
# Show the result
Traffyticket.head(10)

Shape of the dataframe before dropping: (78995, 86)
Shape of the dataframe after dropping: (63292, 86)


,type_,type_PM2.5,type_การเดินทาง,type_กีดขวาง,type_คนจรจัด,type_คลอง,type_ความปลอดภัย,type_ความสะอาด,type_จราจร,type_ต้นไม้,...,district_ห้วยขวาง,district_เมืองลำพูน,district_เมืองสมุทรปราการ,province_กรุงเทพมหานคร,province_จังหวัดกรุงเทพมหานคร,province_จังหวัดนนทบุรี,province_นนทบุรี,province_ลำพูน,province_สมุทรปราการ,duration
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,59.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,28.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,334.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,334.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,158.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,254.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,254.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,94.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,94.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,94.0


In [17]:
print(Traffyticket.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63292 entries, 0 to 47588
Data columns (total 86 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   type_                          63292 non-null  float64
 1   type_PM2.5                     63292 non-null  float64
 2   type_การเดินทาง                63292 non-null  float64
 3   type_กีดขวาง                   63292 non-null  float64
 4   type_คนจรจัด                   63292 non-null  float64
 5   type_คลอง                      63292 non-null  float64
 6   type_ความปลอดภัย               63292 non-null  float64
 7   type_ความสะอาด                 63292 non-null  float64
 8   type_จราจร                     63292 non-null  float64
 9   type_ต้นไม้                    63292 non-null  float64
 10  type_ถนน                       63292 non-null  float64
 11  type_ทางเท้า                   63292 non-null  float64
 12  type_ท่อระบายน้ำ               63292 non-null 

## Train Test Split

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import transforms, utils, datasets

def data_loader(valid_size=0.1,
                random_seed=31,
                batch_size=32,
                shuffle=True):

    # load the dataset
    train_dataset = extract_types(Traffyticket)
    valid_dataset = extract_types(Traffyticket)

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler)
 
    valid_loader = DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler)

    return (train_loader, valid_loader)

In [19]:
y = Traffyticket.pop('duration')
X = Traffyticket


## Creating and Preparing the Model

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

# Define the parameter gridSearch 
param_grid = {
    'criterion':['friedman_mse','squared_error'],
    'max_depth': [2,3,6],
    'min_samples_leaf':[2,5,10],
    'n_estimators':[100,200],
    'random_state': [2023]
}

# Create the model
# model = RandomForestRegressor()
model = GradientBoostingRegressor()

''' BEGIN HYPERPARAMETER TUNING '''
# Create a Grid Search object 
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Train the gridSearch onject on the training data
grid_search.fit(X, y)

# Get the best parameters
best_params = grid_search.best_params_
best_score = -grid_search.best_score_
best_model = grid_search.best_estimator_

# Print the best hyperparameters found by grid search
print("Best hyperparameters: ", best_params)

# Evaluate the pemodelormance of the model on the validation set
test_acc = grid_search.score(X, y)
print("Test accuracy: ", test_acc)

# Print the best hyperparameters found by grid search
print("Best estimators: ", best_model)


''' END HYPERPARAMETER TUNING '''

''' BEGIN FEATURE SELECTION '''
# Use feature selection to select the most important features
selector = SelectFromModel(best_model, threshold='median')
selector.fit(X, y)

# Print the selected feature names
selected_features = X.columns[selector.get_support()]
print('Selected features:', list(selected_features))

''' END FEATURE SELECTION '''

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2023)

## Model Training & Evaluation

In [ ]:
# Train the model on the best parameters
model_best = GradientBoostingRegressor(**best_params)
model_best.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report

# Generate predictions for the testing set
y_pred = model_best.predict(X_test)

# Calculate the MSE and R-squared
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE:", mse)
print("R-squared:", r2)

# Assume y_true and y_pred are the true and predicted labels, respectively
# print(classification_report(y_test, y_pred))

In [ ]:
import pandas as pd

# Assume y_true and y_pred are the true and predicted labels, respectively
df = pd.DataFrame({'true': y_test, 'predicted': y_pred})
print(df.head())
